# Chapter 5. 텍스트 생성

- 트랜스포머 기반 언어 모델은 사람이 작성한 텍스트와 거의 구분이 되지 않는 텍스트를 생성하는 매우 신기한 능력을 발휘

- 어떤 명시적인 감독(supervision)도 없이 텍스트를 생성했다는 점에서 매우 놀라움

- GPT-2와 더 강력한 후속 모델인 GPT-3는 수백만 개의 웹 페이지에서 단순히 다음 단어를 예측하는 방법을 학습해서, 다양한 종류의 입력 프롬프트를 바탕으로 글을 생성할 수 있는 광범위한 기술과 패턴 인식을 습득

- 언어 모델이 사전 훈련하는 동안 덧셈, 단어 철자 배열, 번역 같은 문맥 기반으로 다음 토큰을 예측하는 작업 시퀀스에 어떻게 노출되는지 보여줌

  - 이를 통해 얻은 지깃은 미세튜닝이나 (모델이 충분히 크다면) 추론 과정에서 효율적으로 전이(transfer)됩니다.
  
  - 이런 작업은 사전에 선택된 것이 아니며 파라미터가 수십억 개인 언어 모델을 훈련하는 대규모 말뭉치에 자연스럽게 등장

![그림5-1](image/chapter05_lm-meta-learning.png)

* 현재는 Chat GPT(GPT-3.5, GPT-4 모델로 세계를 광풍에 몰아 넣고 있음, 23년도 5월 기준)

이 장에서는 GPT-2를 사용해 언어 모델의 텍스트 생성 원리를 설명하고, 다양한 디코딩 전략이 생성된 텍스트에 미치는 영향을 살펴봄

## 5.1 일관성 있는 텍스트 생성의 어려움

지금까지 사전 훈련과 지도 학습 기반의 미세 튜닝을 조합해 NLP 문제를 다루는 데 초점을 둡니다. 시퀀스나 토큰 분류 같이 작업에 특화된 헤드에서 예측 생성은 매우 간단합니다. 일련의 로짓을 출력하고 최댓값을 선택하여 예측 클래스를 얻습니다. 또는 소프트맥스 함수를 적용해 클래스별 예측 확률을 얻습니다. 이와 달리 모델의 확률 출력을 텍스트로 변환하려면 **디코딩 방법(decoding method)** 이 필요합니다. 여기에는 텍스트 생성에만 따르는 특수한 어려움이 있습니다.

  - 디코딩은 반복적으로 수행되므로 입력이 모델의 정방향 패스를 한 번 통과할 때보다 많은 계산이 필요함
  
  - 생성된 텍스트의 품질과 다양성은 디코딩 방법과 이에 관련된 하이퍼파라미터에 따라 달라짐
  
디코딩이 어떻게 수행되는지 이해하기 위해 먼저 GPT-2의 사전 훈련 방법과 텍스트 생성에 적용하는 과정을 알아봄





다른 **자기회귀 모델**(autoregressive model) 또는 **코잘 언어 모델**(causal language model)과 마찬가지로, GPT-2는 시작 프롬프트 또는 문맥 시퀀스 $ x = x_1, x_2, ... , x_k$ 가 주어질 때 텍스트에 등장하는 토큰 시퀀스 $y = y_1, y_2, ..., y_t $의 확률 $P(y|x)$를 추정하도록 사전 훈련됨

직접 $P(y|x)$를 추정하기 위해 충분한 훈련 데이터를 획득하기란 불가능하므로, 일반적으로 확률의 연쇄 법칙(chain rule)을 사용해 조건부 확률(conditional problem)의 곱으로 나타냅니다.

$$P(y_1, ... , y_t | x) = \prod_{t=1}^{n}P(y_t | y_{<t}, x) $$

여기서 $y_{<t}$는 시퀀스 $y_1, ... , y_{t-1}$을 간략화한 식입니다. 이 조건부 확률로 자기 회귀 언어 모델링은 문장의 이전 단어가 주어지면 다음 단어를 예측한다는 직관을 얻을 수 있습니다. 앞선 식의 오른쪽 항에 위치한 확률이 이를 설명합니다. 이런 사전 훈련 목표는 과거와 미래의 문맥을 모두 사용해 마스킹된 토큰을 예측하는 BERT와 매우 다릅니다. 

이제 다음 토큰 예측 작업이 임의의 길이를 가진 텍스트 시퀀스를 생성할 때 어떻게 적용할 지 예상됩니다. 아래 그림처럼 "Transformers are the"와 같은 프롬프트로 시작하면 모델은 다음 토큰을 예측합니다. 다음 토큰이 결정되면 이를 프롬프트에 추가해 새로운 입려 시퀀스를 만들고 또 다른 토큰을 생성합니다. 이 과정을 특수한 시퀀스 종료 토큰이나 사전에 정의한 최대 길이에 도달할 때까지 반복합니다.

![그림5-3](image/chapter05_text-generation.png)

**NOTE** 출력 시퀀스가 입력 프롬프트에 따라 결정되므로 이런 종류의 텍스트 생성을 종종 **조건부 텍스트 생성(conditional text generation)** 이라 합니다. 

이 과정의 핵심은 각 타임스텝에서 어떤 토큰을 선택할지 결정하는 디코딩 방법에 있습니다. 언어 모델의 헤드는 각 스텝에서 어휘사전에 있는 토큰마다 로짓 $z_{t, i}$을 생성하므로 소프트맥스를 적용하면 가능한 다음 토큰 $w_i$에 대한 확률 분포를 얻습니다. 

$$ P(y_t = w_i | y_{<t}, x) = softmax(z_{t, i}) $$ 

대부분의 디코딩 방법은 다음과 같은 $\hat{y}$를 선택해 전체적으로 확률이 가장 높은 시퀀스를 찾습니다.

$$\hat{y} = \underset{y}{argmax}P(y|x)$$

직접 $\hat{y}$를 찾으려면 언어 모델로 가능한 모든 시퀀스를 평가해야 합니다. 이런 작업을 합리적인 시간 안에 할수 있는 알고리즘이 없으므로 근사적인 방법을 사용합니다. 이 장에서 이런 근사적인 방법 몇가지를 알아보고 고품질 텍스트를 생성하는 더 똑똑하고 복잡한 알고리즘을 점진적으로 구축하겠습니다.

## 5.2 그리디 서치 디코딩

연속적인 모델 출력에서 이산적인 토큰을 얻는 가장 간단한 디코딩 방법은 각 타임 스텝에서 확률이 가장 높은 토큰을 탐욕적(greedily)으로 선택하는 것입니다. 

$$ \hat{y_{t}} = \underset{y_t}{argmax}P(y_t|y_{<t}, x)$$

그리디 서치 방법을 알아보기 위해 언어 모델링 헤더를 가진 15억개 파라미터의 GPT-2 버전을 로드함

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

2023-06-27 23:01:10.682452: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-27 23:01:10.879858: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 23:01:11.438131: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
!nvidia-smi

Tue Jun 27 23:02:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   40C    P8    22W / 170W |   1705MiB / 12288MiB |     21%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU 메모리 사용량 확인
if device.type == "cuda":
    allocated_memory = torch.cuda.memory_allocated(device=device)
    cached_memory = torch.cuda.memory_reserved(device=device)
    print(f"Allocated Memory: {allocated_memory / 1024 ** 2:.2f} MB")
    print(f"Cached Memory: {cached_memory / 1024 ** 2:.2f} MB")

Allocated Memory: 487.47 MB
Cached Memory: 542.00 MB


그럼 텍스트를 생성함. 허깅페이스 트랜스포머스는 GPT-2 같은 자기회귀 모델을 위해 generatre() 함수를 제공하지만, 작동 방식을 이해하기 위해 직접 이 디코딩 메서드를 구현하겠습니다. 연습 삼아 [그림 5-3]에 있는 반복적인 과정을 그대로 따르겠습니다. "Transformers are the"를 입력 프롬프트로 사용하고 여덟 번의 타임스텝 동안 디코딩을 수행합니다. 각 타임스텝에서 프롬프트의 마지막 토큰에 대한 로짓을 선택하고 소프트맥스를 적용해 확률 분포를 얻습니다. 

그 다음 확률이 가장 높은 토큰을 다음 토큰으로 선택하고 입력 시퀀스에 추가한 후에 이 과정을 다시 반복합니다. 또 대안을 시각적으로 보여주기 위해 타임 스텝마다 확률이 가장 높은 토큰을 다섯개 저장합니다.

In [15]:
import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors = "pt")["input_ids"].to(device) # return_tensors가 파이토치의 텐서 형태로 반환
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids = input_ids)
        
        # 첫 번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용합니다.
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim = -1)
        sorted_ids = torch.argsort(next_token_probs, dim = -1, descending = True)
        
        # 가장 높은 확률의 토큰을 저장
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx + 1}"] = token_choice 
        
        # 예측한 다음 토큰을 입력에 추가합니다. 
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim = -1)
        iterations.append(iteration)
    
pd.DataFrame(iterations)    

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (9.76%),same (2.94%),only (2.87%),best (2.38%),first (1.77%)
1,Transformers are the most,common (22.90%),powerful (6.88%),important (6.32%),popular (3.95%),commonly (2.14%)
2,Transformers are the most common,type (15.06%),types (3.31%),form (1.91%),way (1.89%),and (1.49%)
3,Transformers are the most common type,of (83.13%),in (3.16%),. (1.92%),", (1.63%)",for (0.88%)
4,Transformers are the most common type of,particle (1.55%),object (1.02%),light (0.71%),energy (0.67%),objects (0.66%)
5,Transformers are the most common type of particle,. (14.26%),in (11.57%),that (10.19%),", (9.57%)",accelerator (5.81%)
6,Transformers are the most common type of parti...,They (17.48%),\n (15.19%),The (7.06%),These (3.09%),In (3.07%)
7,Transformers are the most common type of parti...,are (38.78%),have (8.14%),can (7.98%),'re (5.04%),consist (1.57%)


In [17]:
iterations[7] # gpt2라서 책과 결과가 사뭇 다름

{'Input': 'Transformers are the most common type of particle. They',
 'Choice 1': ' are (38.78%)',
 'Choice 2': ' have (8.14%)',
 'Choice 3': ' can (7.98%)',
 'Choice 4': "'re (5.04%)",
 'Choice 5': ' consist (1.57%)'}

각 스텝에서 가능한 다른 문장도 볼 수 있는데, 이는 텍스트 생성의 반복적인 특성을 보여줌. 예측하는데 한 번의 정방향 패스로 충분한 시퀀스 분류 등의 작업과 달리 텍스트 생성은 한 번에 하나의 출력 토큰을 디코딩합니다. 

그리드 서치 구현은 어렵지 않지만 더 복잡한 디코딩 방법을 알아보기 위해 허깅페이스 트랜스포머스에 내장된 generate() 함수를 사용.
앞선 결과를 재현하기 위해 샘플링을 끄고(체크포인트에서 로딩한 모델 설정에 따로 지정되지 않았다면 이 옵션은 기본적으로 False입니다.) 생성 토큰의 개수를 max_new_tokens 매개변수로 지정함

In [18]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens = n_steps, do_sample = False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most common type of particle. They are


**에러 내용**: "어텐션 마스크와 패드 토큰 ID가 설정되지 않았습니다. 그 결과, 예상치 못한 동작을 관찰할 수 있습니다. 안정적인 결과를 얻기 위해 입력의 attention_mask를 전달해 주세요. 열린 끝 생성을 위해 pad_token_id를 eos_token_id:50256로 설정합니다."



이 메시지는 Hugging Face의 Transformers 라이브러리를 사용할 때, 주의해야 하는 사항을 알려주고 있습니다.

1. attention_mask가 설정되지 않았다는 부분: attention_mask는 특정 토큰이 모델에 의해 주의를 받아야 하는지(즉, 토큰이 중요한 정보를 포함하고 있는지) 나타내는 값을 의미합니다. 이 값이 설정되지 않으면, 모델은 토큰 간의 관계를 제대로 파악하지 못할 수 있습니다. 따라서 입력의 attention_mask를 설정하여 안정적인 결과를 얻는 것이 중요합니다.

2. pad_token_id가 설정되지 않았다는 부분: 패딩 토큰은 일반적으로 여러 입력을 동일한 길이로 만들기 위해 사용됩니다. 따라서 pad_token_id를 eos_token_id로 설정한다는 것은 문장의 끝을 나타내는 eos_token_id를 패딩 토큰으로 사용하겠다는 의미입니다. 이것은 열린 끝 생성 (open-end generation)에 대한 기본 설정입니다.

만약 위의 사항을 수정하려면, 당신의 코드에 다음과 같이 attention_mask와 pad_token_id를 명시적으로 설정해주면 됩니다.

```python
# 수정 코드 예시 
input_ids = tokenizer(input_txt, return_tensors="pt", padding=True, truncation=True, max_length=512)["input_ids"].to(device)
attention_mask = tokenizer(input_txt, return_tensors="pt", padding=True, truncation=True, max_length=512)["attention_mask"].to(device)
```




이제 조금 더 재미있는 시도를 실시. OpenAI의 유니콘 기사 재현

앞에서 처럼 토크나이저로 프롬프트를 인코딩하고 긴 텍스트 시퀀스를 생성하기 위해 max_length에 큰 값을 지정

In [19]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, \
in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.\n\n
"""

In [20]:
input_ids = tokenizer(input_txt, return_tensors = "pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length = max_length, do_sample = False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very


그리드 서치 알고리즘은 반복적인 출력 시퀀스를 생성하는 경향이 있어서 뉴스 기사로는 확실히 적절하지 않음.

이는 그리드 서치 알고리즘의 보편적인 문제이며, 이로 인해 최적의 솔루션을 만들어내기 어렵습니다. 

디코딩 측면에서 보면, 확률이 높은 단어가 확률이 낮은 단어보다 먼저 등장하기 때문에 전체적으로 확률이 높은 단어 시퀀스를 생성하지 못하기도 합니다. 

다행히 더 나은 방법이 있습니다. **빔 서치 디코딩(beam search decoding)** 이라는 인기 있는 방법을 알아보겠습니다.

**(Note)** 그리디 서치 디코딩은 다양성이 필요한 텍스트 생성 작업에는 거의 사용되지 않지만, 결정적이고 사실적으로 정확한 출력이 필요한 수식 등의 짧은 문장 생성에는 유용합니다. 이런 작업을 위해 줄바꿈이 있는 "5 + 8 => 13 \n 7 + 2 => 9 \n 1 + 0 =>" 같은 형식의 입력 프롬프트를 제공해 GPT-2의 조건부 생성을 제어할 수 있습니다.

## 5.3 빔 서치 디코딩

빔 서치는 각 스텝에서 확률이 가장 높은 토큰을 디코딩하는 대신, 확률이 가장 높은 상위 b개의 다음 토큰을 추적합니다. 

여기서 b는 **빔(beam)** 또는 **불완전 가설(partial hypothesis)** 의 개수입니다. 

다음 빔 세트는 기존 세트에서 가능한 모든 다음 토큰을 확장하고 확률이 가장 높은 b개의 확장을 선택해 구성합니다. 

이 과정은 최대 길이나 EOS 토큰에 도달할 때까지 반복됩니다. 

  - 확률이 가장 높은 시퀀스는 로그 확률에 따라 b개 빔의 순위를 매겨 선택됨 

**< b=2인 빔 서치 >** 

![그림 5-4](image/chapter05_beam-search.png)

왜 확률이 아니라 로그 확률을 사용해 시퀀스 점수를 매길까요? 

시퀀스의 전체 확률 $P(y_1, y_2, ... , y_t|x)$을 계산하려면 조건부 확률 $P(y_t|y_{<t}, x)$의 곱을 계산해야 하기 때문

각 조건부 확률이 일반적으로 [0, 1] 범위 안에 속한 작은 값이므로 이를 곱해 얻은 전체 확률은 언더플로(underflow)가 쉽게 발생합니다. 

  - 컴퓨터가 이 계산의 결과를 더 이상 정확하게 표현할 수 없다는 의미!
  
  - 예를 들어 t=1024개의 토큰으로 이루어진 시퀀스에서 각 토큰의 확률이 0.5라고 하면 시퀀스 전체 확률은 매우 작은 수가 나옴

In [21]:
0.5 ** 1024

5.562684646268003e-309

이런 값은 수치적으로 불안정해 언더플로가 발생하지만, 로그 확률을 계산하면 이를 피할 수 있음

결합 확률(joint probability)과 조건부 확률(conditional probability)에 로그를 적용하면 로그의 곱셉 규칙에 따라 다음과 같은 식이 나옴

$$logP(y_1, ..., y_t|x) = \sum_{t=1}^{N}logP(y_t|y_{<t}, x)$$

다른 말로 하면, 앞서 본 확률의 곱셈이 로그 확률의 덧셈으로 바뀜

이 방식이 수치적 불안정을 일으킬 확률이 훨씬 적습니다. 예를 들어 이전 예에 대한 로그 확률은 다음과 같이 계산함

In [22]:
import numpy as np 

sum([np.log(0.5)]*1024)

-709.7827128933695

이런 값이 더 다루기 쉬울 뿐 아니라 이 방식은 더 작은 수에도 적용됩니다. 상대적 확률만 비교하면 되므로 로그 확률을 사용해서도 비교가 가능

이제 그리디 서치와 빔 서치로 생성한 텍스트의 로그 확률을 계산해 빔 서치가 전체 확률을 향상하는지 확인하겠습니다. 허깅페이스 트랜스포머스 모델은 입력 토큰이 주어지면 다음 토큰에 대한 정규화되지 않은 로짓을 반환합니다. 따라서 먼저 로짓을 정규화해서 시퀀스의 각 토큰을 위해 전체 어휘사전에 대한 확률 분포를 만듭니다. 그 다음 시퀀스에 있는 토큰 확률만 선택합니다. 이 단계를 구현한 함수는 다음과 같습니다.

In [23]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim = -1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

이 함수는 하나의 토큰에 대한 로그 확률을 제공하므로, 시퀀스의 전체 로그 확률을 얻으려면 각 토큰의 로그 확률을 더합니다. 